In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('himachal_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-chandigarh: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=128.0.6613.85)
Stacktrace:
0   chromedriver                        0x00000001046239a8 cxxbridge1$str$ptr + 1887096
1   chromedriver                        0x000000010461be10 cxxbridge1$str$ptr + 1855456
2   chromedriver                        0x0000000104220be0 cxxbridge1$string$len + 89508
3   chromedriver                        0x000000010420aa68 core::str::slice_error_fail::h6c488016ada29016 + 64036
4   chromedriver                        0x000000010420a9a8 core::str::slice_error_fail::h6c488016ada29016 + 63844
5   chromedriver                        0x000000010429df08 cxxbridge1$string$len + 602316
6   chromedriver                        0x0000000104259a08 cxxbridge1$string$len + 322508
7   chromedriver                        0x00

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


Error occurred while accessing https://www.redbus.in/bus-tickets/dharamshala-to-chandigarh: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=128.0.6613.85)
Stacktrace:
0   chromedriver                        0x00000001011f39a8 cxxbridge1$str$ptr + 1887096
1   chromedriver                        0x00000001011ebe10 cxxbridge1$str$ptr + 1855456
2   chromedriver                        0x0000000100df0be0 cxxbridge1$string$len + 89508
3   chromedriver                        0x0000000100de9bc4 cxxbridge1$string$len + 60808
4   chromedriver                        0x0000000100ddbfe8 cxxbridge1$string$len + 4524
5   chromedriver                        0x0000000100ddd9c4 cxxbridge1$string$len + 11144
6   chromedriver                        0x0000000100ddc39c cxxbridge1$string$len + 5472
7   chromedriver                        0x0000000100ddbc10 cxxbridge1$string$len + 3540
8   chromedriver                        0x0000000100ddbb5c cxxbridge1$string$len + 3360
9   chr

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


Error occurred while accessing page 3: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=128.0.6613.85)
Stacktrace:
0   chromedriver                        0x000000010048b9a8 cxxbridge1$str$ptr + 1887096
1   chromedriver                        0x0000000100483e10 cxxbridge1$str$ptr + 1855456
2   chromedriver                        0x0000000100088be0 cxxbridge1$string$len + 89508
3   chromedriver                        0x0000000100081bc4 cxxbridge1$string$len + 60808
4   chromedriver                        0x0000000100073fe8 cxxbridge1$string$len + 4524
5   chromedriver                        0x00000001000759c4 cxxbridge1$string$len + 11144
6   chromedriver                        0x000000010007439c cxxbridge1$string$len + 5472
7   chromedriver                        0x0000000100073c10 cxxbridge1$string$len + 3540
8   chromedriver                        0x0000000100073b5c cxxbridge1$string$len + 3360
9   chromedriver                        0x0000000100071ef8 c

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


Error occurred while accessing page 4: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=128.0.6613.85)
Stacktrace:
0   chromedriver                        0x0000000102eab9a8 cxxbridge1$str$ptr + 1887096
1   chromedriver                        0x0000000102ea3e10 cxxbridge1$str$ptr + 1855456
2   chromedriver                        0x0000000102aa8be0 cxxbridge1$string$len + 89508
3   chromedriver                        0x0000000102aa1bc4 cxxbridge1$string$len + 60808
4   chromedriver                        0x0000000102a93fe8 cxxbridge1$string$len + 4524
5   chromedriver                        0x0000000102a959c4 cxxbridge1$string$len + 11144
6   chromedriver                        0x0000000102a9439c cxxbridge1$string$len + 5472
7   chromedriver                        0x0000000102a93c10 cxxbridge1$string$len + 3540
8   chromedriver                        0x0000000102a93b5c cxxbridge1$string$len + 3360
9   chromedriver                        0x0000000102a91ef8 c

NameError: name 'driver' is not defined

In [3]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 129,Ordinary 3+2 Non AC Seater,08:50,09h 50m,18:40,2.5,INR 507,37 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 657,Ordinary 3+2 Non AC Seater,11:00,11h 00m,22:00,2.7,INR 507,33 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 515,Ordinary 3+2 Non AC Seater,12:30,10h 20m,22:50,4.2,INR 507,33 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 551,A/C Executive (2+3),15:30,10h 00m,01:30,3.5,INR 622,42 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 525,Ordinary 3+2 Non AC Seater,16:50,10h 10m,03:00,4.2,INR 507,8 Seats available
...,...,...,...,...,...,...,...,...,...,...
245,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...,Ram Dalal Holidays,Volvo A/C Semi Sleeper (2+2),21:00,08h 35m,05:35,4.7,INR 620,21 Seats available
246,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...,Laxmi holidays,Volvo A/C Semi Sleeper (2+2),21:00,09h 10m,06:10,4.4,569,32 Seats available
247,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...,Ram Dalal Holidays,Volvo A/C Semi Sleeper (2+2),22:00,08h 10m,06:10,4.7,INR 860,31 Seats available
248,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...,Everest Holidays,Volvo A/C Semi Sleeper (2+2),22:00,10h 00m,08:00,3.4,674,33 Seats available
